In [57]:
import urllib3
import certifi
import re
import mysql.connector as sql
from datetime import datetime
from bs4 import BeautifulSoup

#Urllib3
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

def generate_date(prev_date):
    from datetime import date, timedelta
    date_list = []
    for i in range(1, prev_date):
        tanggal = date.today() - timedelta(i)
        date_list.append(tanggal)
    
    return date_list

def insert(title, image, url, date):
    db_connection = sql.connect(host='127.0.0.1', database='news_aggregator', user='root', password='')
    cursor = db_connection.cursor()
    
    #Try Except
    try:
        data_table = ("INSERT INTO data (TITLE, IMAGE, URL, DATE) VALUES (%s, %s, %s, %s)")
        data_value = (title, image, url, date)

        cursor.execute(data_table, data_value)
        db_connection.commit()

    except sql.IntegrityError:
        print('duplicate entry')
        
    cursor.close()
    db_connection.close()

In [81]:
month = {'Januari': '01',
         'Februari': '02',
         'Maret': '03',
         'April': '04',
         'Mei': '05',
         'Juni': '06',
         'Juli': '07',
         'Agustus': '08',
         'September': '09',
         'Oktober': '10',
         'November': '11',
         'Desember': '12'
        }

In [86]:
def pull_data_otosia(domain, pagination):
    
    for j in range(1, pagination):
        if j == 1:
            url = domain + '.html'
        else:
            url = domain + str(j) +'.html'
        print(url)
        
        #Get article from website
        req = http.request('GET', url)
        soup = BeautifulSoup(req.data, 'lxml')

        try:
            container = soup.find('div', attrs={'id':'mobart-box-big'})
        except:
            print('container not found')
            break

        try:
            boxes = container.find_all('div', attrs={'class': 'article-index-box'})
        except:
            print('box not found')
            break

        for i in range(0, len(boxes)):
            box = container.find_all('div', attrs={'class': 'article-index-box'})[i]

            #Get Title Article
            try:
                title = box.find('h2').text
            except:
                print('h2.text not found')
                break

            #Get Image
            try:
                image = box.find('img').get('src')
            except:
                print('image not found')
                break

            #Get URL Article
            try:
                url = 'https://www.otosia.com/' + box.find('a').get('href')
            except:
                print('href not found')
                break

            #Get Date
            try:
                dateraw = box.find('span', attrs={'class': 'newsdetail-schedule'}).text
                datestring = re.sub(r""".*, | [0-9]{2}:[0-9]{2}""", '', dateraw)
                datearr = datestring.split(' ')
                
                for key in month:
                    if datearr[1] in key:
                        datearr[1] = month[key]
                        
                datestr = '/'.join(d for d in datearr)
                date = datetime.strptime(datestr, '%d/%m/%Y')
            except:
                print('date not found')
                break
                    
            insert(title, image, url, date)

    return "done"

In [89]:
domain = [
    'https://www.otosia.com/berita/index',
    'https://www.otosia.com/review/index',
    'https://www.otosia.com/tips/index',
]

In [90]:
for url in domain:
    pull_data_otosia(url, 2)

https://www.otosia.com/berita/index.html
https://www.otosia.com/review/index.html
https://www.otosia.com/tips/index.html
duplicate entry
duplicate entry
duplicate entry
duplicate entry
duplicate entry
duplicate entry
duplicate entry
duplicate entry
duplicate entry
duplicate entry
duplicate entry
duplicate entry
